# DX 704 Week 12 Project

This week's project will revisit the email spam classifier project from week 9 using large language model embeddings instead of custom features.


The full project description and a template notebook are available on GitHub: [Project 12 Materials](https://github.com/bu-cds-dx704/dx704-project-12).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Download Data Set

We will be using the Enron spam data set as prepared in this GitHub repository.

https://github.com/MWiechmann/enron_spam_data

You may need to download this differently depending on your environment.

In [1]:
#!wget https://github.com/MWiechmann/enron_spam_data/raw/refs/heads/master/enron_spam_data.zip

In [2]:
import pandas as pd

In [3]:
# pandas can read the zip file directly
enron_spam_data = pd.read_csv("enron_spam_data.zip")
enron_spam_data

,Message ID,Subject,Message,Spam/Ham,Date
0,0,christmas tree farm pictures,NaN,ham,1999-12-10
1,1,"vastar resources , inc .","gary , production from the high island larger ...",ham,1999-12-13
2,2,calpine daily gas nomination,- calpine daily gas nomination 1 . doc,ham,1999-12-14
3,3,re : issue,fyi - see note below - already done .\nstella\...,ham,1999-12-14
4,4,meter 7268 nov allocation,fyi .\n- - - - - - - - - - - - - - - - - - - -...,ham,1999-12-14
...,...,...,...,...,...
33711,33711,= ? iso - 8859 - 1 ? q ? good _ news _ c = eda...,"hello , welcome to gigapharm onlinne shop .\np...",spam,2005-07-29
33712,33712,all prescript medicines are on special . to be...,i got it earlier than expected and it was wrap...,spam,2005-07-29
33713,33713,the next generation online pharmacy .,are you ready to rock on ? let the man in you ...,spam,2005-07-30
33714,33714,bloow in 5 - 10 times the time,learn how to last 5 - 10 times longer in\nbed ...,spam,2005-07-30


In [4]:
(enron_spam_data["Spam/Ham"] == "spam").mean()

0.5092834262664611

## Part 2: Download BERT Model

We will use a pre-trained BERT model to extract embedding vectors as described in lesson 2.1 this week.
Here is sample code to download the model from [Hugging Face](https://huggingface.co/) and extract one vector.
This model is small enough that you can run it with CPU only, but GPUs will be faster if available.

In [5]:
# You may need to install torch and transformers.
# Google Colab will have these installed already.
#
# pip install transformers torch --upgrade

import torch
from transformers import AutoTokenizer, AutoModel

In [6]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
device

'mps'

To download the pre-trained model from Hugging Face, you will need to sign up for a free account with them at https://huggingface.co/join.
Afterwards, get an API token and if you are using Google Colab, save it as a secret named HF_TOKEN.

In [7]:
MODEL_NAME = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
bert_model = AutoModel.from_pretrained(MODEL_NAME)
bert_model.to(device)
bert_model.eval()


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [8]:
@torch.no_grad()
def embed_text(text):
    batch = [text]
    inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt").to(device)
    outputs = bert_model(**inputs)
    # CLS token embedding is the first token's hidden state
    cls_emb = outputs.last_hidden_state[:, 0, :]  # shape: [batch_size, 768]
    return cls_emb.cpu()

In [9]:
v = embed_text("Hi, will you buy my spam?")
v.shape

torch.Size([1, 768])

## Part 3: Create Embedding Vectors

Use BERT to create embeddings for each email in the Enron data set.
You will have to decide how to combine the different columns of the original data set to produce one embedding vector.


Hint: BERT can be run without a GPU, but will be much slower.
Using Google Colab with only a CPU, it runs around 1 embedding per second.
Using Google Colab with the T4 GPU option, it runs around 60 embeddings per second.
Caching is also encouraged to avoid unnecessary reruns.

In [10]:
# YOUR CHANGES HERE
import json
import numpy as np
from tqdm import tqdm

embeddings_data = []

# Loop through all emails
for idx, row in tqdm(enron_spam_data.iterrows(), total=len(enron_spam_data)):
    # Handle missing values and combine text
    subject = str(row['Subject']) if pd.notna(row['Subject']) else ""
    message = str(row['Message']) if pd.notna(row['Message']) else ""
    text = f"{subject} {message}".strip()
    
    # Get embedding from BERT
    emb = embed_text(text)
    
    # Convert to 1D list (shape should be [768])
    emb_list = np.round(emb.squeeze().numpy(), 3).tolist()
    
    # Append to our data list
    embeddings_data.append({
        'Message ID': row['Message ID'],
        'embedding_vector_json': json.dumps(emb_list)
    })

  0%|          | 0/33716 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 33716/33716 [07:53<00:00, 71.17it/s] 


Save your embeddings in a file "embeddings.tsv.gz" with two columns, Message ID and embedding_vector_json, where embedding_vector_json is a JSON-encoded list.
Make sure that embedding_vector_json is a 1 dimensional list, not 2 dimensional.

Hint: don't forget the ".gz" extension indicating gzip compression.
The Pandas `.to_csv` method will automatically add the compression if you save data with a filename ending in ".gz", so you just need to pass it the right filename.

In [11]:
# YOUR CHANGES HERE
embeddings_df = pd.DataFrame(embeddings_data)
embeddings_df.to_csv("embeddings.tsv.gz", sep="\t", index=False, compression="gzip")

print(f"Saved {len(embeddings_df)} embeddings to embeddings.tsv.gz")

Saved 33716 embeddings to embeddings.tsv.gz


Submit "embeddings.tsv.gz" in Gradescope.

## Part 4: Train a Linear Regression

Train an ordinary least squares regression for spam/ham status where spam is treated as target value 1 and ham is treated as target value 0 with your embeddings above as the only input variables.


In [12]:
# YOUR CHANGES HERE
from sklearn.linear_model import LinearRegression

embeddings_df = pd.read_csv("embeddings.tsv.gz", sep="\t", compression="gzip")

X = np.array([json.loads(emb) for emb in embeddings_df['embedding_vector_json']])

y = (enron_spam_data['Spam/Ham'] == 'spam').astype(int).values

model = LinearRegression()
model.fit(X, y)

# extract coefficients
coefficients_df = pd.DataFrame({
    'dim' : range(768),
    'coefficient' : model.coef_
})

Save the coefficients of your linear model in a file "coefficients.tsv" with columns dim and coefficient where dim specifies the offset in the embedding vector (0-767).
Don't worry about the bias term (but your model should still have it).

In [13]:
# YOUR CHANGES HERE
coefficients_df.to_csv("coefficients.tsv", sep="\t", index=False)

print(f"Model trained with {len(X)} samples")
print(f"R² score: {model.score(X, y):.4f}")
print(f"Saved {len(coefficients_df)} coefficients to coefficients.tsv")

Model trained with 33716 samples
R² score: 0.8574
Saved 768 coefficients to coefficients.tsv


Submit "coefficients.tsv" in Gradescope.

## Part 5: Search for Relevant Documents

The file "queries.tsv" specifies 10 queries.
For each of the queries, encode them as a vector, and find the message that is closest using $L_2$.

In [14]:
# YOUR CHANGES HERE

# Load the queries
queries_df = pd.read_csv("queries.tsv", sep="\t")

# Load the embeddings we created earlier
embeddings_df = pd.read_csv("embeddings.tsv.gz", sep="\t", compression="gzip")

# Parse embeddings into numpy array for efficient computation
embedding_matrix = np.array([json.loads(emb) for emb in embeddings_df['embedding_vector_json']])
message_ids = embeddings_df['Message ID'].values

# Process each query
results = []
for idx, row in queries_df.iterrows():
    query_text = row['query']
    
    # Embed the query
    query_emb = embed_text(query_text).squeeze().numpy()
    
    # Compute L2 distance to all documents
    distances = np.linalg.norm(embedding_matrix - query_emb, axis=1)
    
    # Find the closest document (minimum distance)
    closest_idx = np.argmin(distances)
    closest_message_id = message_ids[closest_idx]
    
    results.append({
        'query_id': row['query_id'],
        'query_vector_json': json.dumps(query_emb.tolist()),
        'Message ID': closest_message_id
    })

results_df = pd.DataFrame(results)

Save your results in a file "query-matches.tsv" with columns query_id, query_vector_json, and Message ID.

In [15]:
# YOUR CHANGES HERE
results_df.to_csv("query-matches.tsv", sep="\t", index=False)
print(f"Processed {len(queries_df)} queries")

Processed 10 queries


Submit "query-matches.tsv" in Gradescope.

## Part 6: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 7: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

In [16]:
# write acknowledgements.txt file
with open("acknowledgements.txt", "w") as f:
    f.write("None")